## Imports and datafraem


In [1]:
# DBscan from scratch
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_moons

# Load the data
old_df = pd.read_csv('new_dataset.csv')

# Print the first 5 rows of the data    
old_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,PercentSalaryHike,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.547619,1,1.0,0.715820,Sales,0.000000,0.250000,Life Sciences,0.333333,0.0,...,0.00,0.00,0.0,0.000,0.0,0.00,0.15,0.222222,0.000000,0.344828
1,0.814978,0,0.5,0.185735,Research & Development,0.275862,0.157895,Life Sciences,0.727273,1.0,...,0.92,1.00,0.5,0.025,0.5,0.75,0.25,0.388889,0.066667,0.482759
2,0.613136,1,1.0,0.915934,Research & Development,0.068966,0.380435,Other,1.000000,1.0,...,0.60,0.50,0.0,0.025,0.5,0.75,0.00,0.000000,0.000000,0.000000
3,0.545855,0,0.5,0.928610,Research & Development,0.103448,0.793478,Life Sciences,1.000000,0.0,...,0.44,0.75,0.0,0.025,0.5,0.75,0.20,0.388889,0.200000,0.000000
4,0.444950,0,1.0,0.394188,Research & Development,0.068966,0.173913,Medical,0.181818,1.0,...,0.48,1.00,0.5,0.025,0.5,0.75,0.05,0.111111,0.133333,0.137931


In [2]:
df = old_df.copy()
columns = ['Department', 'EducationField', 'JobRole', 'MaritalStatus', 'OverTime']
#target based encoding, each category of the column will bear the mean
for column in columns:
        if column == 'Department':
                df[column] = df.groupby(column)['JobSatisfaction'].transform('mean')
        elif column == 'EducationField':
                df[column] = df.groupby(column)['HourlyRate'].transform('mean')
        elif column == 'JobRole':
                df[column] = df.groupby(column)['HourlyRate'].transform('mean')
        elif column == 'MaritalStatus':
                df[column] = df.groupby(column)['WorkLifeBalance'].transform('mean')
        elif column == 'OverTime':
                df[column] = df.groupby(column)['YearsSinceLastPromotion'].transform('mean')

df.head()


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,PercentSalaryHike,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.547619,1,1.0,0.715820,0.673292,0.000000,0.250000,0.667702,0.333333,0.0,...,0.00,0.00,0.0,0.000,0.0,0.00,0.15,0.222222,0.000000,0.344828
1,0.814978,0,0.5,0.185735,0.666719,0.275862,0.157895,0.667702,0.727273,1.0,...,0.92,1.00,0.5,0.025,0.5,0.75,0.25,0.388889,0.066667,0.482759
2,0.613136,1,1.0,0.915934,0.666719,0.068966,0.380435,0.623913,1.000000,1.0,...,0.60,0.50,0.0,0.025,0.5,0.75,0.00,0.000000,0.000000,0.000000
3,0.545855,0,0.5,0.928610,0.666719,0.103448,0.793478,0.667702,1.000000,0.0,...,0.44,0.75,0.0,0.025,0.5,0.75,0.20,0.388889,0.200000,0.000000
4,0.444950,0,1.0,0.394188,0.666719,0.068966,0.173913,0.653232,0.181818,1.0,...,0.48,1.00,0.5,0.025,0.5,0.75,0.05,0.111111,0.133333,0.137931


In [3]:
class DBSCAN: 
    def __init__(self, eps, min_pts, data):
        self.eps = eps
        self.min_pts = min_pts
        self.data = data
        self.clusters = []
        self.noise = []
        self.core_pts = []
        self.visited = []
        self.clustered = []
        self.cluster_num = 0
        self.clustered_pts = []
        
    def _distance(self, p1, p2):
        # result = 0
        # for i in range(len(p1)):
        #     if(type(p1[i]) == str or type(p2[i]) == str):
        #         if(p1[i] != p2[i]):
        #             result += 1
        #     else : result += (p1[i] - p2[i]) ** 2
        # return math.sqrt(result)
        return math.sqrt(sum([(a - b) ** 2 for a, b in zip(p1, p2)]))
    
    def _region_query(self, point):
        neighbors = []
        for i in range(len(self.data)):
            if self._distance(point, self.data[i]) < self.eps:
                neighbors.append(i)
        return neighbors
    
    def _expand_cluster(self, point, neighbors):
        self.clusters[self.cluster_num].append(point)
        self.clustered.append(point)
        self.visited.append(point)
        for i in neighbors:
            if i not in self.visited:
                self.visited.append(i)
                new_neighbors = self._region_query(self.data[i])
                if len(new_neighbors) >= self.min_pts:
                    neighbors += new_neighbors
            if i not in self.clustered:
                self.clusters[self.cluster_num].append(i)
                self.clustered.append(i)
                
    def fit(self):
        for i in range(len(self.data)):
            if i not in self.visited:
                self.visited.append(i)
                neighbors = self._region_query(self.data[i])
                if len(neighbors) < self.min_pts:
                    self.noise.append(i)
                else:
                    self.clusters.append([])
                    self._expand_cluster(i, neighbors)
                    self.cluster_num += 1
                    
    def get_clusters(self):
        return self.clusters
    
    def get_noise(self):
        return self.noise



In [4]:
Y = df['Attrition'].values
df = df.drop(['Attrition'], axis=1)
X = df.values

In [5]:
dbscan = DBSCAN(eps=1.2, min_pts=2, data=X)
dbscan.fit()
clusters = dbscan.get_clusters()
noise = dbscan.get_noise()

In [6]:
len(dbscan.get_clusters())

2

In [7]:
dbscan.get_clusters()

[[1,
  27,
  66,
  89,
  111,
  133,
  144,
  188,
  200,
  211,
  262,
  283,
  284,
  331,
  350,
  382,
  399,
  403,
  406,
  452,
  481,
  511,
  528,
  533,
  548,
  634,
  641,
  642,
  718,
  787,
  827,
  846,
  847,
  923,
  1040,
  1155,
  10,
  24,
  31,
  40,
  51,
  56,
  58,
  67,
  68,
  71,
  72,
  73,
  76,
  80,
  84,
  86,
  100,
  101,
  107,
  113,
  128,
  139,
  148,
  152,
  156,
  162,
  170,
  183,
  195,
  202,
  206,
  217,
  222,
  234,
  239,
  247,
  254,
  280,
  288,
  291,
  294,
  296,
  299,
  300,
  306,
  309,
  310,
  321,
  337,
  346,
  355,
  360,
  369,
  373,
  374,
  389,
  404,
  420,
  430,
  432,
  438,
  462,
  468,
  476,
  478,
  480,
  482,
  486,
  491,
  499,
  506,
  510,
  512,
  517,
  520,
  527,
  532,
  534,
  539,
  550,
  555,
  556,
  567,
  568,
  581,
  588,
  590,
  598,
  599,
  604,
  611,
  622,
  625,
  628,
  644,
  655,
  658,
  671,
  675,
  683,
  684,
  725,
  745,
  757,
  766,
  768,
  782,
  790,
  793,
  79

In [8]:
len(dbscan.get_noise())

31

In [9]:
dbscan.get_noise()

[0,
 105,
 110,
 194,
 398,
 411,
 746,
 785,
 875,
 914,
 1024,
 1078,
 1221,
 1301,
 1351,
 1366,
 1414,
 1421,
 1434,
 1437,
 1441,
 1443,
 1444,
 1445,
 1447,
 1462,
 1463,
 1465,
 1466,
 1468,
 1469]

In [10]:
# # sklearn dbscan
# from sklearn.cluster import DBSCAN
# from sklearn.preprocessing import StandardScaler

# X = df.values
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# dbscan = DBSCAN(eps=0.5, min_samples=5)
# clusters = dbscan.fit_predict(X_scaled)

# clusters

In [175]:
class AGNES:
    def __init__(self, n_clusters=2, linkage='average'):
        self.n_clusters = n_clusters
        self.linkage = linkage
        self.labels_ = None
        self.cluster_centers_ = None
        self.n_leaves = None
        self.results = {}
        self.linkage_matrix = []
        self.merge_history = []

    def fit(self, X):
        #X = np.array([self.string_to_numerical(x) for x in X])
        
        self.n_leaves = X.shape[0]
        self.labels_ = np.arange(self.n_leaves)
        self.cluster_centers_ = X.copy()
        self.results[self.n_leaves+1] = [self.labels_.copy(), self.cluster_centers_.copy()]
        self.linkage_matrix = np.zeros((self.n_leaves-1, 4))
        self.merge_history = list(range(X.shape[0]))
        # while self.n_leaves >= self.n_clusters:
        #     self.merge()

        while self.n_leaves >= 1:
            self.merge()
            self.results[self.n_leaves+1] = [self.labels_.copy(), self.cluster_centers_.copy()]
        return self

    def merge(self):
        dist = self.distance(self.cluster_centers_)
        i, j = np.unravel_index(dist.argmin(), dist.shape)
        self.cluster_centers_[i] = self._linkage(i, j)
        
        self.cluster_centers_ = np.delete(self.cluster_centers_, j, axis=0)
        self.labels_[self.labels_ == j] = i
        self.labels_[self.labels_ > j] -= 1
        self.n_leaves -= 1
        # we need to keep track of the clusters, problem is i and j are being lost
        self.linkage_matrix = np.vstack((self.linkage_matrix, [self.merge_history[i], self.merge_history[j], dist[i, j], self.n_leaves]))
        self.merge_history[i] = max(self.merge_history)+1
        self.merge_history[j] = max(self.merge_history)+1
    
    def get_linkage_matrix(self):
        return self.linkage_matrix
    
    def string_to_numerical(self,arr):
        """Convert a string to a numerical value"""
        # Convert the string to a list of ASCII values
        new_arr = []
        for s in arr:
            #print(type(s))
            if (type(s) == str):
                ascii_values = [ord(c) for c in s]
                # Convert the list of ASCII values to a numpy array
                ascii_array = np.array(ascii_values)
                # Return the sum of the array
                new_arr.append(ascii_array.sum())
            else: new_arr.append(s)
        return np.array(new_arr)

    def distance(self, X):
        return np.sqrt(-2 * np.dot(X, X.T) + np.sum(X ** 2, axis=1) + np.sum(X ** 2, axis=1)[:, np.newaxis])

    def _distance(self, p1, p2):
        result = 0
        for i in range(len(p1)):
            if(type(p1[i]) == str or type(p2[i]) == str):
                if(p1[i] != p2[i]):
                    result += 1
            else : result += (p1[i] - p2[i]) ** 2
        return math.sqrt(result)

    def distance2(self, X):
        distances = []
        for i in range(len(X)):
            for j in range(len(X)):
                distances.append(self._distance(X[i], X[j]))
        return np.array(distances).reshape(len(X), len(X))
                
    

    def _linkage(self, i, j):
        if self.linkage == 'average':
            # linkage = []
            # for i in range (len(self.cluster_centers_[i])):
            #     if type(self.cluster_centers_[i][i]) == str:
            #         if self.cluster_centers_[i][i] != self.cluster_centers_[j][i]:
            #             linkage.append(0)
            #         else: linkage.append(1)
            #     else: linkage.append((self.cluster_centers_[i][i] + self.cluster_centers_[j][i]))
            # linkage = np.array(linkage)
            # return linkage / 2
            return (self.cluster_centers_[i] + self.cluster_centers_[j]) / 2
        elif self.linkage == 'single':
            return np.minimum(self.cluster_centers_[i], self.cluster_centers_[j])
        elif self.linkage == 'complete':
            return np.maximum(self.cluster_centers_[i], self.cluster_centers_[j])
        else:
            raise ValueError('Unknown linkage method: {}'.format(self.linkage))

    def predict(self, X):
        return self.labels_

    def fit_predict(self, X):
        self.fit(X)
        return self.predict(X)

    def get_results(self):
        return self.results

In [176]:
agnes = AGNES(n_clusters=2, linkage='average')
res = agnes.fit_predict(X)
res

C:\Users\YsPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in sqrt


array([0, 0, 0, ..., 0, 0, 0])

In [177]:
results = agnes.get_results()

In [178]:
results[2]

[array([0, 0, 0, ..., 1, 1, 1]),
 array([[0.63151344, 1.        , 0.56635773, 0.66671891, 0.86206897,
         0.38043478, 0.66770239, 0.21428571, 0.        , 0.80943241,
         0.47826087, 0.58695652, 0.63654129, 0.47619048, 0.68907875,
         0.17783216, 0.98918475, 0.33333333, 0.15753585, 0.68      ,
         0.75      , 0.5       , 0.1       , 0.33333333, 0.75      ,
         0.27777778, 0.47058824, 0.        , 0.06896552]])]

In [179]:
for each in results[2][0]:
        print(each)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [180]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

def precision(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fp = np.sum((1 - y_true) * y_pred)
    return tp / (tp + fp)

def sensitivity(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fn = np.sum(y_true * (1 - y_pred))
    return tp / (tp + fn)

def specificity(y_true, y_pred):
    tn = np.sum((1 - y_true) * (1 - y_pred))
    fp = np.sum((1 - y_true) * y_pred)
    return tn / (tn + fp)

def f_score(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fn = np.sum(y_true * (1 - y_pred))
    fp = np.sum((1 - y_true) * y_pred)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return 2 * precision * recall / (precision + recall)

# confusion matrix containing true positives, false negatives, false positives and true negatives
def confusion_matrix(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fn = np.sum(y_true * (1 - y_pred))
    fp = np.sum((1 - y_true) * y_pred)
    tn = np.sum((1 - y_true) * (1 - y_pred))
    return np.array([[tp, fn], [fp, tn]])

In [181]:
print(f"accuracy = {accuracy(Y, results[2][0])}")
print(f"precision = {precision(Y, results[2][0])}")
print(f"sensitivity = {sensitivity(Y, results[2][0])}")
print(f"specificity = {specificity(Y, results[2][0])}")
print(f"f_score = {f_score(Y, results[2][0])}")
print(f"confusion_matrix = \n{confusion_matrix(Y, results[2][0])}")

accuracy = 0.3054421768707483
precision = 0.16265060240963855
sensitivity = 0.7974683544303798
specificity = 0.21086780210867803
f_score = 0.270192994996426
confusion_matrix = 
[[189  48]
 [973 260]]


In [182]:
dbscan_res = {}
val = 0
for cluster in dbscan.get_clusters():
        for i in cluster:
                dbscan_res[i] = val
        val += 1

In [183]:
# order dict by key
dbscan_res = {k: dbscan_res[k] for k in sorted(dbscan_res)}
dbscan_res = list(dbscan_res.values())

In [184]:
dbscan_res = np.array(dbscan_res)

In [185]:
print(f"accuracy = {accuracy(Y, dbscan_res)}")
print(f"precision = {precision(Y, dbscan_res)}")
print(f"sensitivity = {sensitivity(Y, dbscan_res)}")
print(f"specificity = {specificity(Y, dbscan_res)}")
print(f"f_score = {f_score(Y, dbscan_res)}")
print(f"confusion_matrix = \n{confusion_matrix(Y, dbscan_res)}")


accuracy = 0.0


C:\Users\YsPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


ValueError: operands could not be broadcast together with shapes (1470,) (1439,) 

In [186]:
# sklearn agglomerative clustering 
from sklearn.cluster import AgglomerativeClustering
agg = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='average')
agg_res = agg.fit_predict(X)

In [187]:
for each in agg_res:
        print(each)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [188]:
print(f"accuracy = {accuracy(Y, agg_res)}")
print(f"precision = {precision(Y, agg_res)}")
print(f"sensitivity = {sensitivity(Y, agg_res)}")
print(f"specificity = {specificity(Y, agg_res)}")
print(f"f_score = {f_score(Y, agg_res)}")
print(f"confusion_matrix = \n{confusion_matrix(Y, agg_res)}")

accuracy = 0.8367346938775511
precision = 0.0
sensitivity = 0.0
specificity = 0.9975669099756691
f_score = nan
confusion_matrix = 
[[   0  237]
 [   3 1230]]


C:\Users\YsPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


### This part is yet to be tested:

In [189]:
def inter_cluster(X, labels):
    # calculate the mean of each cluster
    cluster_means = []
    for i in range(len(np.unique(labels))):
        cluster_means.append(np.mean(X[labels == i], axis=0))
    cluster_means = np.array(cluster_means)
    # calculate the mean of all data points
    all_mean = np.mean(X, axis=0)
    # calculate the inter cluster distance
    inter_cluster_dist = np.sum(np.square(cluster_means - all_mean), axis=1)
    return inter_cluster_dist

In [190]:
inter_cluster(X, results[2][0])

array([0.08980549, 0.00630946])

In [191]:
def intra_cluster(X, labels):
    # calculate the mean of each cluster
    cluster_means = []
    for i in range(len(np.unique(labels))):
        cluster_means.append(np.mean(X[labels == i], axis=0))
    cluster_means = np.array(cluster_means)
    # calculate the intra cluster distance
    intra_cluster_dist = []
    for i in range(len(np.unique(labels))):
        intra_cluster_dist.append(np.sum(np.square(X[labels == i] - cluster_means[i])))
    intra_cluster_dist = np.array(intra_cluster_dist)
    return intra_cluster_dist

In [192]:
intra_cluster(X, results[2][0])

array([ 453.70904645, 1859.08332652])

In [193]:
# import dendrogram plotting
from scipy.cluster.hierarchy import dendrogram


In [194]:
def draw_dendrogram(linkage_matrix):
    # Create linkage matrix and then plot the dendrogram
    dendrogram(linkage_matrix)
    plt.show()

In [195]:
agnes.get_linkage_matrix()

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.27300000e+03, 4.27300000e+03, 1.03238273e-07, 2.00000000e+00],
       [4.40500000e+03, 4.40500000e+03, 1.03238273e-07, 1.00000000e+00],
       [4.40700000e+03, 4.40700000e+03, 5.96046448e-08, 0.00000000e+00]])

In [196]:
#import linkage
from scipy.cluster.hierarchy import linkage

In [197]:
Z = linkage(X, 'average')
Z

array([[1.30000000e+02, 5.66000000e+02, 4.63113429e-01, 2.00000000e+00],
       [2.46000000e+02, 5.05000000e+02, 4.78350163e-01, 2.00000000e+00],
       [4.98000000e+02, 5.65000000e+02, 4.99372278e-01, 2.00000000e+00],
       ...,
       [2.91500000e+03, 2.93500000e+03, 2.07242773e+00, 1.46600000e+03],
       [0.00000000e+00, 2.93600000e+03, 2.16550042e+00, 1.46700000e+03],
       [2.93200000e+03, 2.93700000e+03, 2.72953022e+00, 1.47000000e+03]])

In [198]:
(X.shape, results[2][0].shape)

((1470, 29), (1470,))

In [199]:
draw_dendrogram(agnes.get_linkage_matrix())

ValueError: Linkage 'Z' uses the same cluster more than once.